In [ ]:
using CSV, DataFrames
using PythonPlot
using Statistics

# Load the CSV file

human_agg_long_FullRun = CSV.read("data/human_agg_long_FullRun.csv", DataFrame)
monkey_agg_long_FullRun = CSV.read("data/monkey_agg_long_FullRun.csv", DataFrame)

human_agg_long_FullRun_Filtered = CSV.read("data/human_agg_long_FullRun_filtered.csv", DataFrame)
monkey_agg_long_FullRun_Filtered = CSV.read("data/monkey_agg_long_FullRun_filtered.csv", DataFrame)

# Vertically concatenate human and monkey filtered DataFrames
human_agg_long_FullRun_Filtered.species = fill("human", nrow(human_agg_long_FullRun_Filtered))
monkey_agg_long_FullRun_Filtered.species = fill("monkey", nrow(monkey_agg_long_FullRun_Filtered))
combined_agg_long_FullRun_Filtered = vcat(human_agg_long_FullRun_Filtered, monkey_agg_long_FullRun_Filtered)


In [ ]:
using MixedModels, DataFrames, StatsModels

function fit_acc_glmm(df::DataFrame)
    df = deepcopy(df[df.label .!= "incongruent", :])

    # Response as a proportion with binomial weights
    df.acc = df.y ./ df.n

    # Re-code predictors
    df.pre_c     = (df.pre_ms .- mean(unique(df.pre_ms))) ./ 1000  # scale to seconds, center using mean of unique values
    df.q_c       = df.q .- mean(unique(df.q))               # center using mean of unique values
    df.m_c       = df.m .- mean(unique(df.m))  # center m using mean of unique values
    df.label_c   = ifelse.(df.label .== "congruent_true", 0.5, -0.5)  # effect-coded ±0.5
    df.obs       = 1:nrow(df)                       # for OLRE (optional)

    f = @formula(
        acc ~ 1 +
        label_c * (q_c + pre_c + m_c) +
        (1 + label_c*(m_c + pre_c) | id)
    )

    m = fit(GeneralizedLinearMixedModel, f, df, Binomial(), LogitLink(); wts = df.n)
    return m
end


function fit_acc_glmm_monkey(df::DataFrame)
    df = deepcopy(df[df.label .!= "incongruent", :])

    # Response as a proportion with binomial weights
    df.acc = df.y ./ df.n

    # Re-code predictors
    df.pre_c     = (df.pre_ms .- mean(unique(df.pre_ms))) ./ 1000  # scale to seconds, center using mean of unique values
    df.q_c       = df.q .- mean(unique(df.q))               # center using mean of unique values
    df.m_c       = df.m .- mean(unique(df.m))  # center m using mean of unique values
    df.label_c   = ifelse.(df.label .== "congruent_true", 0.5, -0.5)  # effect-coded ±0.5
    df.obs       = 1:nrow(df)                       # for OLRE (optional)

    f = @formula(
        acc ~ 1 +
        label_c * (pre_c + m_c) +
        (1 + label_c*(m_c + pre_c) | id)
    )

    m = fit(GeneralizedLinearMixedModel, f, df, Binomial(), LogitLink(); wts = df.n)
    return m
end

In [ ]:
hum_filt_m = fit_acc_glmm(human_agg_long_FullRun_Filtered)

In [ ]:
print(hum_filt_m)

In [ ]:
mon_filt_m = fit_acc_glmm_monkey(monkey_agg_long_FullRun_Filtered)

In [ ]:
print(mon_filt_m)